## IOU Testing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PyHa.IsoAutio import generate_automated_labels
from PyHa.statistics import clip_IoU, clip_IoU_orig, clip_IoU_skip, matrix_IoU_Scores
from PyHa.statistics import automated_labeling_statistics

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import time

In [2]:
output_df = {
    "FUNCTION": [],
    "TECHNIQUE": [],
    "DURATION": [],
    "ITERATIONS": [],
    "AVERAGE": []
}
path =  "../acoustic-id-test-data/TEST/"
path2 = "./ScreamingPiha_Manual_Labels.csv"

In [4]:
isolation_parameters = {
    "model" : "microfaune",
    "technique" : "chunk",
    "threshold_type" : "median",
    "threshold_const" : 2.0,
    "threshold_min" : 0.0,
    "chunk_size" : 5.0,
    "window_size" : 2.0
}

manual_df = pd.read_csv(path2)
automated_df = generate_automated_labels(path, isolation_parameters)

def call_IoU(f, automated_df, manual_df):
    start = time.time()
    matrix = f(automated_df, manual_df)
    print('time:', int((time.time() - start) * 10) / 10.0, 'seconds')
    return matrix

# double_scalars error comes from dividing two very small numbers
# this is handled by converting NaN values in the IoU matrix to 0 when it is returned
print("original")
orig = call_IoU(clip_IoU_orig, automated_df, manual_df)
display(matrix_IoU_Scores(orig, manual_df, 0.5))

print("\nslightly edited")
slight = call_IoU(clip_IoU, automated_df, manual_df)
display(matrix_IoU_Scores(slight, manual_df, 0.5))

print("\nlinear algebra")
lin = call_IoU(clip_IoU_edited, automated_df, manual_df)
display(matrix_IoU_Scores(lin, manual_df, 0.5))


C:\Users\saman\OneDrive\Documents\GitHub\PyHa\PyHa\microfaune_package\microfaune\audio.py:23: WavFileWarning: Reached EOF prematurely; finished at 61440 bytes, expected 11782700 bytes from header.
  fs, data = wavfile.read(path)


original
time: 498.2 seconds


,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,./TEST/,ScreamingPiha1.wav,bird,12,242,82,0.1277,0.0472,0.0689



slightly edited
time: 382.8 seconds


,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,./TEST/,ScreamingPiha1.wav,bird,12,242,82,0.1277,0.0472,0.0689



linear algebra


C:\Users\saman\OneDrive\Documents\GitHub\PyHa\PyHa\statistics.py:334: RuntimeWarning: invalid value encountered in double_scalars
  IoU_Matrix[i][j] /= np.sum(np.logical_or(human_arr[i], bot_arr[j]))


time: 193.7 seconds


,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,./TEST/,ScreamingPiha1.wav,bird,12,242,82,0.1277,0.0472,0.0689


## Testing on whole dataset

In [10]:
def call_stats(automated_df, manual_df, stats_type="IoU", threshold=0.5):
    start = time.time()
    stats_df = automated_labeling_statistics(automated_df,manual_df,stats_type,threshold)
    print('time:', int((time.time() - start) * 10) / 10.0, 'seconds')
    return stats_df

stats_df_orig = call_stats(automated_df, manual_df)
stats_df_opt = call_stats(automated_df, manual_df, stats_type="IoU-opt")

Division by zero setting precision, recall, and f1 to zero on ScreamingPiha10.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha2.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha3.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha4.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha6.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha8.wav
time: 107.6 seconds
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha10.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha2.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha3.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha4.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha6.wav
Division by zero setting precision, recall, and f1 to zero on ScreamingPiha8.wav
time: 

### Importing labels

In [8]:
path_to_audio_files = "../acoustic-id-test-data/subset/" 
path_to_ground_truth = "../acoustic-id-test-data/MDD_Xeno_Canto_DSC180_Labels_uniform_3s_binary.csv"

In [9]:
# generate a csv with just the labels for specified files
import os
files = os.listdir(path_to_audio_files)
dscdf = pd.read_csv(path_to_ground_truth)
dscdf = dscdf[dscdf["IN FILE"].isin(files)]
dscdf.to_csv("025_DSC180_subset_manual.csv")
path_to_ground_truth = "025_DSC180_subset_manual.csv"

In [10]:
manual_df = pd.read_csv(path_to_ground_truth)
isolation_parameters = {
    "model" : "microfaune",
    "technique" : "chunk",
    "threshold_type" : "median",
    "threshold_const" : 2.0,
    "threshold_min" : 0.0,
    "chunk_size" : 3.0,
    "window_size" : 2.0
}
automated_df = generate_automated_labels(path_to_audio_files, isolation_parameters)
automated_df.to_csv("025_DSC180_subset_automated")

In [11]:
automated_df

,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,OFFSET,DURATION,MANUAL ID
0,../acoustic-id-test-data/subset/,Aburria-aburri-387212.wav,0,84.48000,44100,0.0,3.00000,bird
1,../acoustic-id-test-data/subset/,Aburria-aburri-387212.wav,0,84.48000,44100,3.0,3.00000,bird
2,../acoustic-id-test-data/subset/,Aburria-aburri-387212.wav,0,84.48000,44100,6.0,3.00000,bird
3,../acoustic-id-test-data/subset/,Aburria-aburri-387212.wav,0,84.48000,44100,9.0,3.00000,bird
4,../acoustic-id-test-data/subset/,Aburria-aburri-387212.wav,0,84.48000,44100,12.0,3.00000,bird
...,...,...,...,...,...,...,...,...
6901,../acoustic-id-test-data/subset/,Zonotrichia-capensis-47546.wav,0,54.93551,44100,42.0,3.00000,bird
6902,../acoustic-id-test-data/subset/,Zonotrichia-capensis-47546.wav,0,54.93551,44100,45.0,3.00000,bird
6903,../acoustic-id-test-data/subset/,Zonotrichia-capensis-47546.wav,0,54.93551,44100,48.0,3.00000,bird
6904,../acoustic-id-test-data/subset/,Zonotrichia-capensis-47546.wav,0,54.93551,44100,51.0,3.00000,bird


In [ ]:
%reload_ext autoreload
def call_stats(automated_df, manual_df, stats_type="IoU", threshold=0.5):
    start = time.time()
    stats_df = automated_labeling_statistics(automated_df,manual_df,stats_type = "IoU",threshold = 0.5)
    print('time:', int((time.time() - start) * 10) / 10.0, 'seconds')
    return stats_df

stats_df_bn_orig = call_stats(automated_df, manual_df)

In [ ]:
stats_df_bn_opt = call_stats(automated_df, manual_df, stats_type="IoU-opt")

## Verification

In [10]:
print(len(slight), 'x', len(slight[0]))
print(len(lin), 'x', len(lin[0]))

254 x 93
254 x 93


In [12]:
print(slight)

[[0.4259 0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.17   ... 0.     0.     0.    ]
 [0.2    0.     0.     ... 0.     0.     0.    ]
 ...
 [0.     0.     0.     ... 0.0228 0.2939 0.    ]
 [0.     0.     0.     ... 0.     0.4122 0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]]


In [37]:
slight.tolist()
print(*slight, sep="\n")

[0.4259 0.     0.     0.     0.     0.     0.     0.4259 0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.4259 0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.4259 0.
 0.     0.     0.     0.     0.4259 0.     0.4259 0.     0.     0.4259
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.4259 0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.4259 0.     0.     0.
 0.     0.     0.     0.     0.     0.4259 0.     0.     0.     0.
 0.     0.     0.    ]
[0.     0.     0.17   0.     0.     0.     0.     0.     0.     0.17
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.17   0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.17   0.     0.     0.     0.     0.     0.     0.     0.2389 0.
 0.     0.17   0.     0.     0.  

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.     0.     0.     0.     0.     0.     0.7505 0.     0.     0.
 0.     0.     0.     0.7505 0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.7505 0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.7505 0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.7505 0.     0.     0.     0.
 0.     0.     0.     0.     0

In [28]:
lin.tolist()
print(*lin, sep="\n")

[0.4259 0.     0.     0.     0.     0.     0.     0.4259 0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.4259 0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.4259 0.
 0.     0.     0.     0.     0.4259 0.     0.4259 0.     0.     0.4259
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.4259 0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.4259 0.     0.     0.
 0.     0.     0.     0.     0.     0.4259 0.     0.     0.     0.
 0.     0.     0.    ]
[0.     0.     0.17   0.     0.     0.     0.     0.     0.     0.17
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.17   0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.17   0.     0.     0.     0.     0.     0.     0.     0.2389 0.
 0.     0.17   0.     0.     0.  

 0.    0.342 0.    0.    0.    0.    0.    0.    0.   ]
[0.048  0.     0.     0.     0.     0.     0.     0.048  0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.048  0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.048  0.
 0.     0.     0.     0.     0.048  0.     0.048  0.     0.     0.048
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.048  0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.3448 0.048  0.     0.     0.
 0.     0.     0.     0.     0.     0.048  0.     0.     0.     0.
 0.     0.     0.    ]
[0.118 0.    0.    0.    0.    0.    0.    0.118 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.118 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.118 0.    0.    0.    0.    0.    0.118 0.    0.118 0.
 0.    0.118 0.    0.    0.    0.  

In [45]:
# prints the coords of any differences
import numpy as np
lin = np.nan_to_num(lin)
differences = slight - lin
d = np.nonzero(differences)
d0 = d[0].tolist()
d1 = d[1].tolist()

for i in range(len(d0)):
    print(d0[i], d1[i])